In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv("G:/Education/Machine Learning/ML_CSV_FILES/Movies Dataset/ml-1m/movies.dat", sep = '::', engine = 'python', encoding = 'latin-1')
users = pd.read_csv("G:/Education/Machine Learning/ML_CSV_FILES/Movies Dataset/ml-1m/users.dat", sep = '::', engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv("G:/Education/Machine Learning/ML_CSV_FILES/Movies Dataset/ml-1m/ratings.dat", sep = '::', engine = 'python',encoding = 'latin-1')

# Preparing training and test set

* .base be the training set  
* .test be the test set
* delimiter acts as a seperator to data set. \t tells the next data to go on to next tab

In [3]:
training_set = pd.read_csv("G:/Education/Machine Learning/ML_CSV_FILES/Movies Dataset/ml-100k/u1.base", delimiter = '\t')

##### Converting training set in to an array
The reason to convert in to array is, in order to use the torch and tensor functions to it.

In [4]:
training_set = np.array(training_set, dtype = 'int')

In [5]:
test_set= pd.read_csv("G:/Education/Machine Learning/ML_CSV_FILES/Movies Dataset/ml-100k/u1.test", delimiter = '\t')

In [6]:
test_set = np.array(test_set, dtype = 'int')

### Getting Number of Users & Movies

This helps in providing the count of number of movies and number of users

In [7]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [8]:
nb_users

943

In [9]:
nb_movies

1682

### Converting the data in to  array with users in lines and movies in columns

In [10]:
def convert(data):
    new_data = [] # To create list #
    for id_users in range(1, nb_users + 1): # as we know thst nb_users are 943, the indexing starts from 0 in that time the last variable 943 will get lost, so we added +1 to the list #
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings # This helps in sending the code to the first index value starting at zero rather than sending it to one #
        new_data.append(list(ratings))
    return new_data

In [11]:
training_set = convert(training_set)
test_set = convert(test_set)

#### Converting the data in to Torch tensors.
we can even use tensorflow tensors or numpy tensors.

In [12]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

#### Converting the ratings into binary ratings 1(liked), 0(unliked)

In [13]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Building Neural Network

* nv = number of visisble nodes, nh = number of hidden nodes
* W = weights (upper case because we are considering all weights)
* p_h_given_v = probability of hidden nodes with given visible nodes
* p_v_given_h = probability of visible nodes with given hidden nodes
* v0 = visible nodes iteration starting at zero
* vk = visible nodes iteration ending at k
* t = transpose of matrix
* ph = probability of hidden nodes

In [14]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [15]:
nv = len(training_set[0])
nh = 100
batch_size = 100 # the batch size can be tunable till 943, because there are 943 lists in data frame. We can change it till we get the best tuned response #
rbm = RBM(nv, nh)

## Training RBM

In [16]:
nb_epoch = 10

In [17]:
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3713)
epoch: 2 loss: tensor(0.2534)
epoch: 3 loss: tensor(0.2538)
epoch: 4 loss: tensor(0.2354)
epoch: 5 loss: tensor(0.2478)
epoch: 6 loss: tensor(0.2481)
epoch: 7 loss: tensor(0.2491)
epoch: 8 loss: tensor(0.2481)
epoch: 9 loss: tensor(0.2454)
epoch: 10 loss: tensor(0.2466)


### Testing RBM

In [18]:
test_loss = 0
s = 0.

In [19]:
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2422)
